In [14]:
from netCDF4 import Dataset
import numpy as np
from calendar import monthrange
import datetime
from cartoplot import cartoplot
import tqdm
import pickle
import mask
import xarray as xr

winter_months = ['01','02','03','04','10','11','12']

In [11]:
# Get the data for the grid

lon = mask.get('lon')
lat = mask.get('lat')

In [18]:
source_dir = "/home/robbie/Dropbox/SM_Thickness/data/SnowModel/"
output_dir = '/home/robbie/Dropbox/SM_Thickness/data/SnowModel/monthly_means/361/'


def grid_monthly():
    
    # Read the daily density and depth data from the NetCDF files

    E5_path_den = f"{source_dir)sden_E5.nc"
    E5_path_dep = f"{source_dir)snod_E5.nc"

    M2_path_den = f"{source_dir)sden_M2.nc"
    M2_path_dep = f"{source_dir)snod_M2.nc"
    
    
    E5_dep_data=Dataset(E5_path_dep,"r")
    E5_den_data=Dataset(E5_path_den,"r")

    M2_dep_data=Dataset(M2_path_dep,"r")
    M2_den_data=Dataset(M2_path_den,"r")


    # What should the start day be to start 1/1/1981?

    monthlist_1980 = ['08','09','10','11','12']
    monthlist_full = ['01','02','03','04','05','06','07','08','09','10','11','12']
    monthlist_2018 = ['01','02','03','04','05','06','07']

    year_list = list(range(1980,2019))
    
    start_num = 0

    for year in year_list:
        
        print(year)
        
        if year == 1980:
            monthlist = monthlist_1980
        elif year == 2018:
            monthlist = monthlist_2018
        else:
            monthlist = monthlist_full
        
        for month in monthlist:

            # Get number of days in month

            month_len = monthrange(year,int(month))[1]

            # Get the day numbers for the start and end of the month

            print(month,year)

            E5_month_den_array = E5_den_data['sden'][start_num:start_num+month_len]
            E5_month_dep_array = E5_dep_data['snod'][start_num:start_num+month_len]

            M2_month_den_array = M2_den_data['sden'][start_num:start_num+month_len]
            M2_month_dep_array = M2_dep_data['snod'][start_num:start_num+month_len]                

            ave_month_den_array = np.mean([E5_month_den_array, M2_month_den_array], axis=0)
            ave_month_dep_array = np.mean([E5_month_dep_array, M2_month_dep_array], axis=0)
            ave_month_SWE_array = np.multiply(ave_month_den_array,ave_month_dep_array)

            #####################################################################

            # Code for making files for each month with daily data

#             if year > 2002 and year < 2010 and month in winter_months:
                
#                 day_list = list(range(1,month_len+1))                
                
#                 ds = xr.Dataset( data_vars={'depth':(['t','x','y'],ave_month_dep_array),
#                                             'density':(['t','x','y'],ave_month_den_array),
#                                             'SWE':(['t','x','y'],ave_month_SWE_array)},

#                                  coords =  {'lon':(['x','y'],mask.get('lon')),
#                                             'lat':(['x','y'],mask.get('lat')),
#                                             'day':(['t'],day_list)})

#                 data_dir = '/home/robbie/Dropbox/SM_Thickness/data/SnowModel/daily_data/'

#                 ds.to_netcdf(f'{data_dir}{year}{month}.nc','w')

#             # Average the daily values in the month

            #####################################################################


            E5_month_den = np.nanmean(E5_month_den_array,axis=0)
            E5_month_dep = np.nanmean(E5_month_dep_array,axis=0)
            E5_month_SWE = np.multiply(E5_month_den,E5_month_dep)

            M2_month_den = np.nanmean(M2_month_den_array,axis=0)
            M2_month_dep = np.nanmean(M2_month_dep_array,axis=0)
            M2_month_SWE = np.multiply(M2_month_den,M2_month_dep)

            ave_month_den = np.mean([E5_month_den, M2_month_den], axis=0)
            ave_month_dep = np.mean([E5_month_dep, M2_month_dep], axis=0)            
            ave_month_SWE = np.multiply(ave_month_den,ave_month_dep)


            ds = xr.Dataset( data_vars =   {'E5 density':(['x','y'],E5_month_den),
                                            'E5 depth'  :(['x','y'],E5_month_dep),
                                            'E5 SWE'    :(['x','y'],E5_month_SWE),
                                            'M2 density':(['x','y'],M2_month_den),
                                            'M2 SWE'    :(['x','y'],M2_month_SWE),
                                            'M2 depth'  :(['x','y'],M2_month_dep),
                                            'density'   :(['x','y'],ave_month_den),
                                            'depth'     :(['x','y'],ave_month_dep),
                                            'SWE'       :(['x','y'],ave_month_SWE)},

                             coords={'lon':(['x','y'],lon),
                                     'lat':(['x','y'],lat)})

            ds.attrs['data_name'] = 'Monthly mean snow depth and density fields from SnowModel-LG'

            ds.attrs['citation'] = 'Data from SnowModel-LG (author Glen Liston), unpublished.'

            ds.attrs['year'] = f"""{year}"""
            ds.attrs['month'] = f"""{month}"""

            ds.attrs['python author'] = """Robbie Mallett wrote this python code. If there's a problem with it, 
                                            email him at robbie.mallett.17@ucl.ac.uk"""

            ds.to_netcdf(f'{output_dir}{year}{month}.nc','w')

            # Update starting day

            start_num += month_len

    print(start_num)

# Do the update

In [6]:
den = '/home/robbie/Dropbox/SM_Thickness/data/SnowModel/netcdf_from_gdat/SM_sden_MERRA2_ease_01Aug2018-31Jul2021.nc'
dep = '/home/robbie/Dropbox/SM_Thickness/data/SnowModel/netcdf_from_gdat/SM_snod_MERRA2_ease_01Aug2018-31Jul2021.nc'

M2den = Dataset('/media/robbie/TOSHIBA_EXT/snowmodel_nc_data/SM_sden_MERRA2_01Aug1980-31Jul2018_v01.nc')
M2dep = Dataset('/media/robbie/TOSHIBA_EXT/snowmodel_nc_data/SM_snod_MERRA2_01Aug1980-31Jul2018_v01.nc')
E5den = Dataset('/media/robbie/TOSHIBA_EXT/snowmodel_nc_data/SM_sden_ERA5_01Aug1980-31Jul2018_v01.nc')
E5dep = Dataset('/media/robbie/TOSHIBA_EXT/snowmodel_nc_data/SM_snod_ERA5_01Aug1980-31Jul2018_v01.nc')

output_dir = '/home/robbie/Dropbox/SM_Thickness/data/SnowModel/monthly_means/361/'

# Make time axis

In [9]:
dts = [datetime.datetime(1,1,1) + datetime.timedelta(hours=int(h)) for h in M2den['time']]

/tmp/ipykernel_25835/1699274443.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dts = [datetime.datetime(1,1,1) + datetime.timedelta(hours=int(h)) for h in M2den['time']]


In [15]:
monthlist_1980 = np.arange(8,13)
monthlist_full = np.arange(1,13)
monthlist_2018 = np.arange(1,8)

for year in tqdm.trange(2002,2018):
    
    if year == 1980:
        monthlist = monthlist_1980
    elif year == 2018:
        monthlist = monthlist_2018
    else:
        monthlist = monthlist_full
        
    
    for month in np.arange(1,13):
        
        start = datetime.datetime(year,month,1)
        if month !=12:
            end = datetime.datetime(year,month+1,1)
        else:
            end = datetime.datetime(year+1,1,1)

        val_dts = [dt for (idx, dt) in enumerate(dts) if (dt>=start) and (dt<end)]
        rel_inds = [idx for (idx, dt) in enumerate(dts) if (dt>=start) and (dt<end)]

        M2_month_den = np.nanmean(M2den['sden'][rel_inds[0]:rel_inds[-1]],axis=0)
        M2_month_dep = np.nanmean(M2dep['snod'][rel_inds[0]:rel_inds[-1]],axis=0)
        M2_month_SWE = np.multiply(M2_month_den,M2_month_dep)
        
        
        E5_month_den = np.nanmean(E5den['sden'][rel_inds[0]:rel_inds[-1]],axis=0)
        E5_month_dep = np.nanmean(E5dep['snod'][rel_inds[0]:rel_inds[-1]],axis=0)
        E5_month_SWE = np.multiply(E5_month_den,E5_month_dep)

        ave_month_den = np.mean([E5_month_den, M2_month_den], axis=0)
        ave_month_dep = np.mean([E5_month_dep, M2_month_dep], axis=0)            
        ave_month_SWE = np.multiply(ave_month_den,ave_month_dep)

        ds = xr.Dataset( data_vars =   {
                                        'E5 density':(['x','y'],E5_month_den),
                                        'E5 depth'  :(['x','y'],E5_month_dep),
                                        'E5 SWE'    :(['x','y'],E5_month_SWE),
                                        'M2 density':(['x','y'],M2_month_den),
                                        'M2 SWE'    :(['x','y'],M2_month_SWE),
                                        'M2 depth'  :(['x','y'],M2_month_dep),
                                        'density'   :(['x','y'],ave_month_den),
                                        'depth'     :(['x','y'],ave_month_dep),
                                        'SWE'       :(['x','y'],ave_month_SWE),
        },

                         coords={'lon':(['x','y'],lon),
                                 'lat':(['x','y'],lat)})

        ds.attrs['data_name'] = 'Monthly mean snow depth and density fields from SnowModel-LG'

        ds.attrs['citation'] = 'Data from SnowModel-LG (author Glen Liston), unpublished.'

        ds.attrs['year'] = f"""{year}"""
        ds.attrs['month'] = f"""{month}"""

        ds.attrs['python author'] = """Robbie Mallett wrote this python code. If there's a problem with it, 
                                        email him at robbie.mallett.17@ucl.ac.uk"""

        ds.to_netcdf(f'{output_dir}{year}{month}.nc','w')

  0%|                                                    | 0/16 [00:00<?, ?it/s]/tmp/ipykernel_25835/1133076179.py:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  M2_month_den = np.nanmean(M2den['sden'][rel_inds[0]:rel_inds[-1]],axis=0)
/tmp/ipykernel_25835/1133076179.py:15: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  M2_month_dep = np.nanmean(

In [19]:
grid_monthly()

1980
08 1980
09 1980
10 1980
11 1980
12 1980
1981
01 1981
02 1981
03 1981
04 1981
05 1981
06 1981
07 1981
08 1981
09 1981
10 1981
11 1981
12 1981
1982
01 1982
02 1982
03 1982
04 1982
05 1982
06 1982
07 1982
08 1982
09 1982
10 1982
11 1982
12 1982
1983
01 1983
02 1983
03 1983
04 1983
05 1983
06 1983
07 1983
08 1983
09 1983
10 1983
11 1983
12 1983
1984
01 1984
02 1984
03 1984
04 1984
05 1984
06 1984
07 1984
08 1984
09 1984
10 1984
11 1984
12 1984
1985
01 1985
02 1985
03 1985
04 1985
05 1985
06 1985
07 1985
08 1985
09 1985
10 1985
11 1985
12 1985
1986
01 1986
02 1986
03 1986
04 1986
05 1986
06 1986
07 1986
08 1986
09 1986
10 1986
11 1986
12 1986
1987
01 1987
02 1987
03 1987
04 1987
05 1987
06 1987
07 1987
08 1987
09 1987
10 1987
11 1987
12 1987
1988
01 1988
02 1988
03 1988
04 1988
05 1988
06 1988
07 1988
08 1988
09 1988
10 1988
11 1988
12 1988
1989
01 1989
02 1989
03 1989
04 1989
05 1989
06 1989
07 1989
08 1989
09 1989
10 1989
11 1989
12 1989
1990
01 1990
02 1990
03 1990
04 1990
05 1990
0